In [2]:
from google.colab import files

uploaded = files.upload()

import pandas as pd
import io

# 파일명은 실제 업로드한 파일명으로 변경해주세요.
df = pd.read_csv(io.BytesIO(uploaded['reservation.csv']))

# 데이터프레임 확인
print(df.head())

Saving reservation.csv to reservation.csv
   _id  mbNo  fellow dogName  dogSize          reserveDate  oneWay  waitTime  \
0   25    46       1     마로리     웰시코기  2021-08-30 13:00:00       1         0   
1   29    45       1      바니       푸들  2021-08-26 14:10:00       1         0   
2   30    48       1      반디    포메라니안  2021-09-01 14:00:00       0         0   
3   31    27       1      두부  강아지(믹스)  2021-09-04 13:00:00       1         0   
4   34     4       1      코코      말티즈  2021-09-01 10:00:00       1         0   

         deptAddress         destAddress  status  driverMbNo chargeType  \
0  서울 강남구 언주로172길 28     서울 서초구 사평대로 205       3         5.0        NaN   
1    서울 서초구 서래로8길 18  서울 강남구 봉은사로 지하 601       3        64.0        NaN   
2  서울 강남구 봉은사로57길 63   서울 서초구 서초대로27길 33       3        64.0        NaN   
3     서울 강남구 언주로 123       서울 강남구 청담동 46       3        64.0        NaN   
4     서울 강남구 신사동 667   서울 강남구 역삼동 708-31       3        64.0        NaN   

             createdAt    

In [3]:
# 'destAddress' 열만 선택하여 새로운 데이터프레임 생성
df_destAddress = df[['destAddress']]

# 새로운 CSV 파일로 저장
df_destAddress.to_csv('destAddress.csv', index=False)

print("destAddress.csv file has been saved.")


destAddress.csv file has been saved.


In [ ]:
files.download('destAddress.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
# 데이터프레임 확인
print(df_destAddress.head())

# 'destAddress' 열을 리스트로 변환
destAddress_list = df_destAddress['destAddress'].tolist()

# 리스트를 문자열로 변환 (각 항목을 개행 문자로 구분)
destAddress_str = '\n'.join(destAddress_list)

# 텍스트 파일로 저장
with open('destAddress.txt', 'w') as f:
    f.write(destAddress_str)

print("destAddress.txt file has been saved.")

files.download('destAddress.txt')


          destAddress
0     서울 서초구 사평대로 205
1  서울 강남구 봉은사로 지하 601
2   서울 서초구 서초대로27길 33
3       서울 강남구 청담동 46
4   서울 강남구 역삼동 708-31
destAddress.txt file has been saved.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
def address_to_place(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    key = "key" # 카카오 rest api key
    headers = {"Authorization": f"KakaoAK {key}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    documents = response.json().get("documents", [])
    if not documents:  # "documents"가 비어 있는 경우
        return address, None
    result = documents[0]
    x = result["x"]
    y = result["y"]
    place_name = result.get("place_name")  # place_name을 가져옵니다. 없는 경우 None을 반환합니다.
    return address, x, y, place_name


In [ ]:
import requests

def address_to_place(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    key = "key"  # 카카오 rest api key
    headers = {"Authorization": f"KakaoAK {key}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    documents = response.json().get("documents", [])
    if not documents:  # "documents"가 비어 있는 경우
        return address, None, None, None
    result = documents[0]
    x = result["x"]
    y = result["y"]
    place_name = result.get("place_name")  # place_name을 가져옵니다. 없는 경우 None을 반환합니다.
    return address, x, y, place_name

with open('destAddress.txt', 'r', encoding='utf-8') as f:
    addresses = [line.strip() for line in f]

with open('destAddress_output.txt', 'w', encoding='utf-8') as file:
    for address in addresses:
        address_info = address_to_place(address)
        file.write(', '.join(str(i) if i else 'None' for i in address_info) + '\n')
